Здесь представлен код. Подробные выводы можно посмотреть в readme

Полиномиальная регрессия (данные предобработаны)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[["stop_id", "next_stop","stop_distance", "lon", "lng"]], data[["timeBetweenStop"]], test_size=0.33, random_state=42)

# полиномиальная регресиия
from sklearn.linear_model import LinearRegression


# Fitting Polynomial Regression to the dataset 
from sklearn.preprocessing import PolynomialFeatures 

poly = PolynomialFeatures(degree = 2) 
X_poly = poly.fit_transform(X_train[["stop_distance", "lon", "lng"]]) 

poly.fit(X_poly, y_train) 
lin2 = LinearRegression().fit(X_poly, y_train) 

X_poly_test = poly.fit_transform(X_test[["stop_distance", "lon", "lng"]]) 


y_pred = lin2.predict(X_poly_test)
X_poly_validate = poly.fit_transform(validate[["stop_distance", "lon", "lng"]])

y_validate = lin2.predict(X_poly_validate)


fill = pd.DataFrame(index=allStopWithTime.index[allStopWithTime.isnull().any(axis=1)], data= y_validate,columns=["timeBetweenStop"])
resultsData = allStopWithTime.fillna(fill)
resultsData = resultsData[["stop_id", "next_stop", "timeBetweenStop", "hasStopThisInteval"]]

allBusWithTime = pd.merge(allBus, resultsData, how="left" ,on=['stop_id', 'next_stop'])

Нейронная сеть (данные предобработаны)

In [ ]:
import numpy as np
def reject_outliers(data, m=1):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

if max(indexes) - min(indexes) > 5:
    print(reject_outliers(indexes))

In [ ]:
X_train_orignal,X_test_orignal, y_train_orignal, y_test_orignal = X_train, X_test, y_train, y_test

x_train0 = X_train[["lon", "lng", "stop_distance"]]
x_test0 = X_test[["lon", "lng", "stop_distance"]]

In [ ]:
import torch
x_train_torch = torch.tensor(x_train0.values)
y_train_torch = torch.tensor(y_train.values)
x_test_torch = torch.tensor(x_test0.values)
y_test_torch = torch.tensor(y_test.values)

x_train_torch.unsqueeze_(1)
y_train_torch.unsqueeze_(1)
x_test_torch.unsqueeze_(1)
y_test_torch.unsqueeze_(1)

In [ ]:
X_train = torch.FloatTensor(x_train0.values)
X_test = torch.FloatTensor(x_test0.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

In [ ]:
import torch
import random

mBest = 10000

class RegressionNet(torch.nn.Module):
    random.seed(10)
    def __init__(self, n_hidden_neurons):
        super(RegressionNet, self).__init__()
        self.fc1 = torch.nn.Linear(5, n_hidden_neurons)
        self.act1 = torch.nn.ReLU()
        #self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        #self.act2 = torch.nn.ReLU()
        #self.fc3 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        #self.act3 = torch.nn.ReLu()
        self.fc4 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.act4 = torch.nn.ReLU()
        self.fc5 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.act5 = torch.nn.ReLU()
        self.fc6 = torch.nn.Linear(n_hidden_neurons, 1)

    def forward(self, x):
      x = self.fc1(x)
      x = self.act1(x)
      #x = self.fc2(x)
      #x = self.act2(x)
      #x = self.fc3(x)
      #x = self.act3(x)
      x = self.fc4(x) 
      x = self.act4(x)
      x = self.fc5(x)
      x = self.act5(x)
      x = self.fc6(x)
      return x


    def inference(self, x):
      x = self.forward(x)
      x = self.sm(x)
      return x

net = RegressionNet(107)


optimizer = torch.optim.Adam(net.parameters(), lr=0.01)
#optimizer = torch.optim.Adadelta(net.parameters(), lr=0.01, rho=0.9, eps=1e-06, weight_decay=0)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss = nn.MSELoss()

def loss(pred, target):
    abses = abs(pred - target)
    return abses.mean()
'''
#loss = torch.nn.CrossEntropyLoss()
'''
for epoch_index in range(5000):
    optimizer.zero_grad()

    y_pred = net.forward(X_train)
    loss_value = loss(y_pred, y_train)
    loss_value.backward()
    optimizer.step()
    # make backward
    # make step

    if epoch_index % 100 == 0:
      test_preds = net.forward(X_test)
      #test_preds = test_preds.argmax(dim=1)
      MSE = torch.sum((test_preds - y_test) ** 2, axis=0) / float(X_test.shape[0] - X_test.shape[1])

      print(MSE) 


def metric(pred, target):
    return (pred - target).abs().mean()

print(metric(net.forward(X_test), y_test).item())

'''
def predict(net, x, y):
    y_pred = net.forward(x)

    plt.plot(x.numpy(), y.numpy(), 'o', label='Groud truth')
    plt.plot(x.numpy(), y_pred.data.numpy(), 'o', c='r', label='Prediction');
    plt.legend(loc='upper left')
    plt.xlabel('$x$')
    plt.ylabel('$y$')
'''

m = metric(net.forward(X_test), y_test).item()

if m < mBest:
    mBest = metric(net.forward(X_test), y_test).item()
    iBest = i

print(i, mBest)

Посчитаем также рассчтояние до центра города ( как еще один параметр)

In [ ]:
coordsOfCenter = (59.93394044473929, 30.324399338624293)

In [ ]:
import geopy.distance
from geopy import distance
from geopy import Point
p1 = Point("59.93394044473929 30.324399338624293")
data["distanceToCenter"] = distance.distance(data["coordinates"],p1).km

In [ ]:
def distancer(row):
    coords_1 = (row['lon'], row['lng'])
    coords_2 = ("59.93394044473929, 30.324399338624293")
    return geopy.distance.VincentyDistance(coords_1, coords_2).km

data['distanceToCenter'] = data.apply(distancer, axis=1)

также можно добавить бинарную переменную едет автобус в центр или из центра


In [ ]:
# также можно добавить бинарную переменную едет автобус в центр или из центра
def distancerNext(row):
    coords_1 = (row['lonNext'], row['lngNext'])
    #print(pd.isna(row['lonNext']))
    if pd.isna(row['lonNext']) != True:
        coords_2 = ("59.93394044473929, 30.324399338624293")
        return geopy.distance.VincentyDistance(coords_1, coords_2).km
    else:
        return None

df0['distanceToCenterFromNext'] = df0.apply(distancerNext, axis=1)

import numpy as np
df0["inCenter"] = np.where((df0.distanceToCenter > df0.distanceToCenterFromNext),1 , 0)


In [ ]:
Рассмотрим данные полученные

In [ ]:
# гистограмма
df0[["lon", "lng", "timeBetweenStop","distanceToCenter","lonNext","lngNext",  "inCenter" ]].hist();


In [ ]:
# матрица рассеяния
from pandas.plotting import scatter_matrix
scatter_matrix(df0[["lon", "lng", "timeBetweenStop","distanceToCenter" ]], alpha=0.2, figsize=(6, 6), diagonal='kde');

In [ ]:
# рассмотрим данные о времени движения в конкретный промежэуток времени для автобуса направленного в центр и из центраъ
# с поомощью критерия Манна Уитни оценим является ли матенматическое ождидлание времени движения автобуса из центра и в центр одинаковым 
x = df0[df0['inCenter'] == 1]['timeBetweenStop']
y = df0[df0['inCenter'] == 0]['timeBetweenStop']
x.name, y.name = 'inCenter', 'fromCenter'

import matplotlib.pyplot as plt

x[x < 600].hist(alpha=0.5, bins = 50)
y[y < 600].hist(alpha=0.5, bins = 50)
plt.axvline(x.mean(), color='red', alpha=0.8, linestyle='dashed')
plt.axvline(y.mean(), color='blue', alpha=0.8, linestyle='dashed')
plt.legend([x.name, y.name])
plt.title("Test of equality of averages")

from scipy import stats
res = stats.mannwhitneyu(x[x < 2000], y[y < 2000])
print('p-value: {0}'.format(res[1]))
# очень маленькое p-value. значит у нас есть основания отвергнуть гипотезу. Разница средних значений в выборках неслучайна
# время для транпорта из центра и в центр отличаются

In [ ]:
# применим тест Шапиро-Уилка для опроеджеллинения нормальности распредлений

res = stats.shapiro(x)
print('p-value: {0}'.format(res[1])) # распредление нормальное
res = stats.shapiro(y)
print('p-value: {0}'.format(res[1])) # распредление  нормальное

In [ ]:
# рассмотрим такой жу тест но для данных автобьусов находящихся в радиусе 5 км от центра

In [ ]:
x = df0[df0['distanceToCenter'] < 5 ]['timeBetweenStop']
y = df0[df0['distanceToCenter'] > 5 ]['timeBetweenStop']
x.name, y.name = 'center', 'notCenter'

x[x < 1000].hist(alpha=0.5, bins = 50)
y[y < 1000].hist(alpha=0.5, bins = 50)
plt.axvline(x.mean(), color='red', alpha=0.8, linestyle='dashed')
plt.axvline(y.mean(), color='blue', alpha=0.8, linestyle='dashed')
plt.legend([x.name, y.name])
plt.title("Test of equality of averages")

from scipy import stats
res = stats.mannwhitneyu(x, y)
print('p-value: {0}'.format(res[1]))
# очень маленькое p-value. значит у нас есть основания отвергнуть гипотезу. Разница средних значений в выборках неслучайна
# время для транпорта из центра и в центр отличаются

In [ ]:
# рассмотрим ращличия для севера и юга
x = df0[(df0['lon'] < 59.93394044473929) & (df0['lng'] < 30.324399338624293)]['timeBetweenStop']
y = df0[(df0['lon'] > 59.93394044473929) & (df0['lng'] > 30.324399338624293)]['timeBetweenStop']
x.name, y.name = 'south', 'north'

x.hist(alpha=0.5, bins = 50)
y.hist(alpha=0.5, bins = 50)
plt.axvline(x.mean(), color='red', alpha=0.8, linestyle='dashed')
plt.axvline(y.mean(), color='blue', alpha=0.8, linestyle='dashed')
plt.legend([x.name, y.name])
plt.title("Test of equality of averages")

from scipy import stats
res = stats.mannwhitneyu(x[x < 1000], y[y < 1000], alternative = "greater")
print('p-value: {0}'.format(res[1]))

# очень маленькое p-value. значит у нас есть основания отвергнуть гипотезу. Разница средних значений в выборках неслучайна
# время для транпорта из центра и в центр отличаются

In [ ]:
df0 = df[["lon", "lng", "timeBetweenStop", "distanceToCenter", "inCenter",  "stop_distance"]]
df0["speed"] = df0["stop_distance"] / df0["timeBetweenStop"]

df[df.timeBetweenStop < 400].boxplot(column = "timeBetweenStop")

In [ ]:
# сделаем кластерны йаннализ по скорости движения транспората по городу
df1.columns
df1 = df0[["lon", "lng", "inCenter", "speed"]]

# Стандартизируем данные
from sklearn import preprocessing
norm = preprocessing.StandardScaler()
norm.fit(df1)
X = norm.transform(df1)
#X = df0[df0.timeBetweenStop < 400].dropna().dropna()

from sklearn.cluster import KMeans

model = KMeans(n_clusters=5, random_state=42)
model.fit(X)
df1['cluster'] = model.labels_
df1.groupby('cluster').mean()

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

K = range(1, 11)
models = [KMeans(n_clusters=k, random_state=42).fit(X) for k in K]
dist = [model.inertia_ for model in models]

# Plot the elbow
plt.plot(K, dist, marker='o')
plt.xlabel('k')
plt.ylabel('Sum of distances')
plt.title('The Elbow Method showing the optimal k')
plt.show() # по моему мнению оптимальные варинт 5 кластеров

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
link = linkage(X, 'ward', 'euclidean')
dn = dendrogram(link)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[["stop_distance", "lon", "lng", "distanceToCenter", "inCenter"]], data[["timeBetweenStop"]], test_size=0.33, random_state=42)
data0 = data[["stop_distance", "lon", "lng", "distanceToCenter", "lonNext", "lngNext", "inCenter", "timeBetweenStop"]]

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
print('R^2: {0}'.format(model.score(X_train, y_train))) # результат на обучающей выборке

coef = pd.DataFrame(zip(['intercept'] + X_train.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
model.coef_.tolist()


# рассмотрим коэфииценты, по цоэффицентьам можно сказать как изменения независмой переменной влияет на зависимую

In [ ]:
# матрица корреляций
data0.corr()


# коэффиценты регресии
from scipy import stats

def regression_coef(model, X, y):
    a = model.coef_.tolist()[0]
    a.insert(0, model.intercept_[0])
    coef = pd.DataFrame(zip(['intercept'] + X_train.columns.tolist(), a),
                    columns=['predictor', 'coef'])

    X1 = np.append(np.ones((len(X),1)), X, axis=1)
    b = np.append(model.intercept_, model.coef_)
    MSE = np.sum((model.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    var_b = MSE.timeBetweenStop * (np.linalg.inv(np.dot(X1.T, X1)).diagonal())
    sd_b = np.sqrt(var_b)
    t = b / sd_b
    coef['pvalue'] = [2 * (1 - stats.t.cdf(np.abs(i), (len(X1) - 1))) for i in t]
    return coef
regression_coef(model, X_train, y_train)

In [ ]:
# полиномиальная
from sklearn.linear_model import LinearRegression


# Fitting Polynomial Regression to the dataset 
from sklearn.preprocessing import PolynomialFeatures 

X_train = X_train[["stop_distance", "lon", "lng", "distanceToCenter", "inCenter"]]
X_test = X_test[["stop_distance", "lon", "lng", "distanceToCenter", "inCenter"]]

poly = PolynomialFeatures(degree = 3) 

X_poly = poly.fit_transform(X_train) 

poly.fit(X_poly, y_train) 

lin2 = LinearRegression().fit(X_poly, y_train) 

X_poly_test = poly.fit_transform(X_test) 


y_pred = lin2.predict(X_poly_test)

MSE = np.sum((y_pred - y_test) ** 2, axis=0) / float(X_test.shape[0] - X_test.shape[1])
print(MSE)
from sklearn.metrics import r2_score
print('R^2: {0}'.format(r2_score(y_test, y_pred)))

In [ ]:
# дерево решений 
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=42,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='mse',
                               # максимальная глубина дерева
                               max_depth=3,
                               # минимальное число элементов в узле для разбиения (может быть долей)
                               min_samples_split=5,
                               # минимальное число элементов в листе (может быть долей)
                               min_samples_leaf=5,
                               # минимальное значение дельты impurity
                               # min_impurity_decrease=0,
                               # веса для классов (можно дополнительно штрафовать за ошибку в нужных классах).
                               # поддерживает опцию 'balanced'.
                  
                               # предварительная сортировка.
                               # ускоряет обучение на данных небольшого размера или с ограниченной глубиной дерева.
                               # иначе замедляет обучение.
                               presort=False)
model.fit(X_train, y_train)


# рассмоотрим коэффициенты
pd.DataFrame({'feature': X_train.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
# случайный лес

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=42,
                              # опции, относящиеся к отдельным деревьям такаие же, как в tree.DecisionTreeClassifier
                              # число деревьев в лесу
                              n_estimators=6,
                              # функция для impurity ('gini' или 'entropy')
                              criterion='mse',
                              max_depth=5,
                              #min_samples_split = 2,
                              #min_samples_leaf = 1
                              )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
MSE = np.sum((y_pred - y_test['timeBetweenStop'].tolist()) ** 2, axis=0) / float(X_test.shape[0] - X_test.shape[1])

print(MSE)
print('R^2: {0}'.format(r2_score(y_test, y_pred)))

y_pred = model.predict(X_test)

pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
# градиентный бустинг


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import classification_report

model = GradientBoostingRegressor(random_state=42,
                                  # Число деревьев, участвующих в приближении решения
                                  n_estimators=32,
                                  # Максимальная глубина каждого дерева
                                  max_depth=3,
                                  # Параметр, уменьшающий переобучение, являющемся весом отдельного дерева.
                                  # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                  learning_rate=0.1
                                  # Есть и другие параметры, уменьшающие размер дерева,
                                  # такие же как у DecisionTreeClassifier
                                  )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#print (classification_report(y_test, y_pred))
# classification_report считает 3 метрики - точность, полноту и ф1
MSE = np.sum((y_pred - y_test['timeBetweenStop'].tolist()) ** 2, axis=0) / float(X_test.shape[0] - X_test.shape[1])

print(MSE)
print('R^2: {0}'.format(r2_score(y_test, y_pred)))


fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)


In [ ]:
# xgboost. Также подбираем параметры когда модель выдает лучшие результаты
from xgboost import XGBRegressor

iBest = 0
jBest = 0
kBest = 0
mseBest = 10000
for i in range (2, 600, 5):
    for j in range (2, 30):
        for k in [0.1]:
            model = XGBRegressor(seed=42,
                            n_estimators=i,
                            max_depth=j,
                            learning_rate=k)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            MSE = np.sum((y_pred - y_test['timeBetweenStop'].tolist()) ** 2, axis=0) / float(X_test.shape[0] - X_test.shape[1])

            if MSE < mseBest:
                
                mseBest = MSE
                iBest = i
                jBest = j
                kBest = k

print(mseBest)
print('R^2: {0}'.format(r2_score(y_test, y_pred)))



In [ ]:
# catboost
from catboost import CatBoostRegressor

'''
X_train = df.drop('income', axis=1)
y_train = df['income']

X_test = test.drop('income', axis=1)
y_test = test['income']

cat_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
cat_features = [i for i, x in enumerate(df.columns) if x in cat_features]

'''

model = CatBoostRegressor(random_seed=42,
                          iterations=112,
                          depth=5,
                          learning_rate=0.1,
                          verbose=False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
#print (classification_report(y_test, y_pred))
MSE = np.sum((y_pred - y_test['timeBetweenStop'].tolist()) ** 2, axis=0) / float(X_test.shape[0] - X_test.shape[1])

print('R^2: {0}'.format(r2_score(y_test, y_pred)))


fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)